# MAP 536 - Python for Data Science - Predicting Cyclist Traffic in Paris

In [14]:
# %% [markdown]
# # MAP 536 - Python for Data Science - Predicting Cyclist Traffic in Paris

# %% [code]
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import numpy as np
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import TimeSeriesSplit

# %% [code]
## + Weather data
# Load training and testing datasets & remove unnecessary cols
train_data = pd.read_parquet("train.parquet")
test_data = pd.read_parquet("test.parquet")
train_data.drop(columns=['counter_name', 'site_name','counter_id', 'counter_installation_date', 'counter_technical_id', 'site_id'], inplace=True)
test_data.drop(columns=['counter_name', 'site_name','counter_id', 'counter_installation_date', 'counter_technical_id', 'site_id'], inplace=True)

#Load weather dataset and remove irrelevant columns
weather_data = pd.read_csv("hourly-weather-data.csv")
weather_data = weather_data.drop(columns=['name', 'dew', 'preciptype','uvindex','icon','stations', 'sealevelpressure', 'winddir', 'sealevelpressure', 'solarradiation', 'solarenergy', 'severerisk'])

# convert to datetime to merge them properly
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

#merge with weather data
merged_train_data = pd.merge(train_data, weather_data, left_on='date', right_on='datetime', how='inner')
merged_test_data = pd.merge(test_data, weather_data, left_on='date', right_on='datetime', how='inner')
merged_train_data = merged_train_data.drop(columns=['datetime'])
merged_test_data = merged_test_data.drop(columns=['datetime'])


## School Holidays + Weather data
# import the holiday dataset
schoolholiday_data = pd.read_csv("fr-calendrier.csv", delimiter=';')
schoolholiday_data = schoolholiday_data[schoolholiday_data['zones'] == 'Zone C'] # Zone C is Paris
schoolholiday_data = schoolholiday_data.drop(columns=['description','location','annee_scolaire', 'zones'])

# Convert the date strings to datetime objects
schoolholiday_data['start_date'] = pd.to_datetime(schoolholiday_data['start_date'], utc=True).dt.date
schoolholiday_data['end_date'] = pd.to_datetime(schoolholiday_data['end_date'],utc=True).dt.date

# Generate a set of unique dates for each range in a row
unique_dates = set()
for _, row in schoolholiday_data.iterrows():
    unique_dates.update(pd.date_range(start=row['start_date'], end=row['end_date']))

# Convert the set back to a list and create a DataFrame
unique_dates_list = sorted(list(unique_dates)) 
schoolholiday_data = pd.DataFrame({'Date': unique_dates_list})

# merge with rest of the data
merged_train = pd.merge(merged_train_data, schoolholiday_data, left_on='date', right_on='Date', how='left')
merged_train['Date'] = merged_train['Date'].apply(lambda x: 0 if pd.isna(x) else 1)
merged_train.rename(columns={'Date': 'is_holiday'}, inplace=True)

merged_test = pd.merge(merged_test_data, schoolholiday_data, left_on='date', right_on='Date', how='left')
merged_test['Date'] = merged_test['Date'].apply(lambda x: 0 if pd.isna(x) else 1)
merged_test.rename(columns={'Date': 'is_holiday'}, inplace=True)


## School Holidays + Weather data + strike data
# import the strike dataset
from datetime import datetime  # Import the datetime class from the datetime module

# strike dates for public transport in Paris, retrieved from: https://www.cestlagreve.fr/calendrier/?lieu=74&secteur=14&mois=1&annee=2022
strike_dates = {'Date': [datetime(2020, 9, 17), datetime(2020, 12, 14), datetime(2020, 12, 16),
                        datetime(2021, 1, 21), datetime(2021, 2, 4), datetime(2021, 2, 15),
                        datetime(2021, 5, 21), datetime(2021, 6, 1), datetime(2021, 10, 5),
                        datetime(2021, 11, 17)]}

strike_data = pd.DataFrame(strike_dates)
strike_data

# Convert the date strings to datetime objects
strike_data['Date'] = pd.to_datetime(strike_data['Date'])


# merge with rest of the data
merged_train = pd.merge(merged_train, strike_data, left_on='date', right_on='Date', how='left')
merged_train['Date'] = merged_train['Date'].apply(lambda x: 0 if pd.isna(x) else 1)
merged_train.rename(columns={'Date': 'is_strike'}, inplace=True)

merged_test = pd.merge(merged_test, strike_data, left_on='date', right_on='Date', how='left')
merged_test['Date'] = merged_test['Date'].apply(lambda x: 0 if pd.isna(x) else 1)
merged_test.rename(columns={'Date': 'is_strike'}, inplace=True)
merged_test.head()


## School Holidays + Weather data + Strike data + Lockdown data
lockdown_data = pd.read_csv("lockdown-data.csv")
lockdown_data['datetime'] = pd.to_datetime(lockdown_data['datetime'])
merged_train['date'] = pd.to_datetime(merged_train['date'])
merged_test['date'] = pd.to_datetime(merged_test['date'])

merged_train = pd.merge(merged_train, lockdown_data, left_on='date', right_on='datetime', how='left')
merged_train = merged_train.drop(columns=['datetime'])
merged_test = pd.merge(merged_test, lockdown_data, left_on='date', right_on='datetime', how='left')
merged_test = merged_test.drop(columns=['datetime'])


## Encode the dates
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    X['date'] = pd.to_datetime(X['date'])
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour
    return X

merged_train = _encode_dates(merged_train)
merged_test = _encode_dates(merged_test)

# drop the date column
X_merged_train = merged_train.drop(columns=['bike_count', 'log_bike_count', 'conditions'])
Y_merged_train = merged_train['log_bike_count']

X_merged_test = merged_test.drop(columns=['bike_count', 'log_bike_count', 'conditions'])
Y_merged_test = merged_test['log_bike_count']
X_merged_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41608 entries, 0 to 41607
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               41608 non-null  datetime64[us]
 1   latitude           41608 non-null  float64       
 2   longitude          41608 non-null  float64       
 3   temp               41608 non-null  float64       
 4   feelslike          41608 non-null  float64       
 5   humidity           41608 non-null  float64       
 6   precip             41608 non-null  float64       
 7   precipprob         41608 non-null  int64         
 8   snow               41608 non-null  float64       
 9   snowdepth          41608 non-null  float64       
 10  windgust           41608 non-null  float64       
 11  windspeed          41608 non-null  float64       
 12  cloudcover         41608 non-null  float64       
 13  visibility         41608 non-null  float64       
 14  is_hol

In [ ]:
## Encode the dates
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    X['date'] = pd.to_datetime(X['date'])
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour
    return X

merged_train = _encode_dates(merged_train)
merged_test = _encode_dates(merged_test)

# drop the date column
X_merged_train = merged_train.drop(columns=['bike_count', 'log_bike_count', 'conditions'])
Y_merged_train = merged_train['log_bike_count']

X_merged_test = merged_test.drop(columns=['conditions'])
#Y_merged_test = merged_test['log_bike_count']
#X_merged_test.info()



import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

# Function to extract date-related features
def encode_dates(df):
    if 'date' in df.columns:
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
        df['weekday'] = df['date'].dt.weekday
        df['hour'] = df['date'].dt.hour
        df = df.drop(columns=['date'])
    return df


if 'date' in X_merged_train.columns:
    X_merged_train = encode_dates(X_merged_train)

if 'date' in X_merged_test.columns:
    X_merged_test = encode_dates(X_merged_test)

# Define numerical features to be scaled
numerical_features = ['latitude', 'longitude', 'temp', 'feelslike', 'humidity', 
                      'precip', 'precipprob', 'snow', 'snowdepth', 'windgust', 
                      'windspeed', 'cloudcover', 'visibility',
                      'year', 'month', 'day', 'weekday', 'hour']

# Preprocessor for scaling
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features)
    ])

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, 
                               learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10))
])

# Fit the pipeline on the training data
pipeline.fit(X_merged_train, Y_merged_train)

# Predict on the test data
y_pred = pipeline.predict(X_merged_test)

# Prepare the submission
results = pd.DataFrame({
    'Id': np.arange(y_pred.shape[0]),
    'log_bike_count': y_pred
})

# Save the results to a CSV file
results.to_csv("submission.csv", index=False)